# Building the Bot Service Backend and Frontend Applications

In the previous notebook, we assembled all the functions and code required to create a robust Agent/Bot. Depending on the user's question, this Agent/Bot searches for answers in the available sources and tools.

However, the question arises: **"How can we integrate this code into a Bot backend application capable of supporting multiple channel deployments?"** Our ideal scenario involves building the bot once and deploying it across various channels such as MS Teams, Web Chat, Slack, Alexa, Outlook, WhatsApp, Line, Facebook, and more.


To achieve this, we need a service that not only aids in building the bot as an API but also facilitates the exposure of this API to multiple channels. This service is known as <u>Azure Bot Framework</u>.

In this notebook, you will learn how to deploy the code you have developed so far as a Bot API using the Bot Framework API and Service.<br>

## What is the Azure Bot Framework and Bot Service?

Microsoft Bot Framework and Azure Bot Service are a collection of libraries, tools, and services that let you build, test, deploy, and manage intelligent bots.

Bots are often implemented as a web application, hosted in Azure and using APIs to send and receive messages.

Azure Bot Service and the Bot Framework include:

- Bot Framework SDKs for developing bots in C#, JavaScript, Python, or Java.
- CLI tools for help with end-to-end bot development.
- Bot Connector Service, which relays messages and events between bots and channels.
- Azure resources for bot management and configuration.

So, in order to build our application we would use the **Bot Framework Python SDK to build the Web API**, and the **Bot Service to connect our API to mutiple channels**.

## Architecture

The image below shows:
1) An Azure Web App hostoing the Bot API
2) Azure Bot Service providing the connection between the Bot API, Channels and Application Insights

![Botframework](./images/Bot-Framework.png)

# Backend - Azure Web App - Bot API

All the functions and prompts used in the prior notebook to create our brain Agent are located in `utils.py` and `prompts.py` respectively.
So, what needs to be done is, basically, to do the same we did in the prior notebook but within the Bot Framework Python SDK classes.

Within the `apps/backend/botservice/` folder, you will find three files: `app.py`, `bot.py` and `config.py`.
- `app.py`: is the entrance main point to the application.
- `bot.py`: is where our OpenAI-related code resides 
- `config.py`: declares the PORT the API will listen from and the App Service Principal var names

We would only need to deal with `bot.py`, here is where all the logic code related to your Azure OpenAI application lives.

in `apps/backend/botservice/README.md` you will find all the instructions on how to:
1) Deploy the Azure web services: Azure Web App and Azure Bot Service
2) Zip the code and uploaded to the Azure Web App
3) Test your Bot API using the Bot Service in the Azure portal

GO AHEAD NOW AND FOLLOW THE INSTRUCTIONS in `apps/backend/botservice/README.md`

# Frontend - Azure Web App - Streamlit 

Once you have the Backend Bot API app running and succesfully tested using the Bot Service Azure portal , we can proceed now to build a sample UI.

In `apps/frontend/` folder you will find the files necesary to build a simple Streamlit application that will have:

1) <u>A Search Interface</u>: Using `utils.py` and `prompts.py` and streamlit functions
2) <u>A BotService Chat Interface</u>: Using the Bot Service Web Chat javascript library we can render the WebChat Channel inside Streamlit as an html component
3) <u>A FastAPI Chat Interface</u>: Using LangServe/FastAPI as backend, we use streamlit components to provide a streaming chat interface (MORE ON THIS LATER)

Notice that in (1) the logic code is running in the Frontend Web App, however in (2) and (3) the logic code is running in the Backend Bot API and the Frontend is just using the WebChat channel from the Bot Service.

GO AHEAD NOW AND FOLLOW THE INSTRUCTIONS in `apps/frontend/README.md`

# (optional) Build and deploy with GitHub Actions

if you don't want to build and deploy all the components manually, a GitHub automated CI pipeline is provided in `.github/workflows/main_gptsmartsearch_apps.yml`. Some notes about the CI pipeline design:
- It uses a "branch per environment approach". The deploy environment name is computed at 'runtime' based on a branch/env-name mapping logic in the "set environment for branch" step (line 29). The current implemented logic maps everything to a dev like environment. Therefore on each git push on the `main branch` the pipeline is triggered trying to deploy to an environment called `Development`. For more info about GitHub environments and how to set specific env variables and secrets read [here](https://docs.github.com/en/actions/deployment/targeting-different-environments/using-environments-for-deployment).
- GitHub environment variables and secrets are used to configure development environment specific configuration. They need to be configured manually in github repository settings:
    - `secrets.AZUREAPPSERVICE_X_PUBLISHPROFILE` is used to store the azure app service publish profile configuration securely.
    - `vars.AZURE_WEBAPP_X_NAME` is used to store the azure web app resource name generated during infra arm deployment.
- Python dependencies installation is disabled during build phase as azure web apps are currently configured with SCM_DO_BUILD_DURING_DEPLOYMENT. There is an env properties that can be used to activate dependencies resolution during build job. Just set `DO_BUILD_DURING_DEPLOYMENT : false `.

To properly configure automated build and deploy for both backend and frontend components follow below steps:
 
 1. Go to your forked repository in GitHub and create an [environment]((https://docs.github.com/en/actions/deployment/targeting-different-environments/using-environments-for-deployment)) called 'Development' (yes this exact name; don't change it). If you want to change the environment name, add new branches and environments  or change the current branch/env mapping you can do that, but make sure to change the pipeline code accordingly in `.github/workflows/main_gptsmartsearch_apps.yml` (starting line 29)
 2. Create 'Development' environment [secrets](https://docs.github.com/en/actions/security-guides/encrypted-secrets#creating-encrypted-secrets-for-a-repository) for both frontend and backend azure web apps [publish profiles]((https://learn.microsoft.com/en-us/visualstudio/azure/how-to-get-publish-profile-from-azure-app-service?view=vs-2022)). You'll need to copy paste the xml content from the .PublishSettings file into the secret value:
     - Create a secret with name `AZUREAPPSERVICE_BACKEND_PUBLISHPROFILE` and set the Value field to publish profile of the azure web app you created using 'Deploy to Azure' button in `apps/backend/README.md`
     - Create a secret with name `AZUREAPPSERVICE_FRONTEND_PUBLISHPROFILE` and set the Value field to publish profile of the azure web app you created using 'Deploy to Azure' button in `apps/frontend/README.md`
3. Create 'Development' environment [variables](https://docs.github.com/en/actions/learn-github-actions/variables#creating-configuration-variables-for-an-environment) for both frontend and backend azure web app resource names:
    - Create a variable with name `AZURE_WEBAPP_BACKEND_NAME` and set the Value field to the azure web app resource name you created using 'Deploy to Azure' button in `apps/backend/README.md`
    - Create a variable with name `AZURE_WEBAPP_FRONTEND_NAME` and set the Value field to the azure web app resource name you created using 'Deploy to Azure' button in `apps/frontend/README.md`
4. For each commit you push check the status of the triggered pipeline in the GitHub Actions tab, you should see a pipeline has been triggered for the specific commit. If everything is ok you should see green checkmark on both build and deploy jobs in the pipeline detail like below:

![pipeline success](./images/github-actions-pipeline-success.png)


# Reference

- https://learn.microsoft.com/en-us/azure/bot-service/bot-service-overview?view=azure-bot-service-4.0
- https://github.com/microsoft/botbuilder-python/tree/main
- https://github.com/microsoft/BotFramework-WebChat/tree/master